In [2]:
import pandas as pd
import numpy as np
from itertools import product
import math

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
# ============================================================
# 1. LOAD DATA
# ============================================================
df = pd.read_csv('Adult_clean.csv')

# take only needed columns for model
nodes = [
    'age_bin', 'race', 'sex', 'native-country', 'education',
    'occupation', 'workclass', 'hours_bin', 'marital-status', 'income'
]

df = df[nodes].copy()

for col in df.columns:
    df[col] = df[col].astype(str)

df

,age_bin,race,sex,native-country,education,occupation,workclass,hours_bin,marital-status,income
0,Adult,White,Male,US,Bachelors,Adm-clerical,Gov,Full-time,Never-married,<=50K
1,Middle,White,Male,US,Bachelors,Exec-managerial,Private/Self-emp,Part-time,Married-civ-spouse,<=50K
2,Adult,White,Male,US,HS-grad,Handlers-cleaners,Private/Self-emp,Full-time,Divorced,<=50K
3,Middle,Black,Male,US,HS-or-less,Handlers-cleaners,Private/Self-emp,Full-time,Married-civ-spouse,<=50K
4,Adult,Black,Female,Other,Bachelors,Prof-specialty,Private/Self-emp,Full-time,Married-civ-spouse,<=50K
...,...,...,...,...,...,...,...,...,...,...
48837,Adult,White,Female,US,Bachelors,Prof-specialty,Private/Self-emp,Full-time,Divorced,<=50K
48838,Middle,Black,Male,US,HS-grad,Missing,Missing,Full-time,Widowed,<=50K
48839,Adult,White,Male,US,Bachelors,Prof-specialty,Private/Self-emp,Long,Married-civ-spouse,<=50K
48840,Adult,Asian-Pac-Islander,Male,US,Bachelors,Adm-clerical,Private/Self-emp,Full-time,Divorced,<=50K


In [5]:
# ============================================================
# 2. SPLIT TRAINING AND TESTING SET (80/20)
# ============================================================

# shuffled data
shuffled = df.sample(frac=1.0, random_state=42).reset_index(drop=True)

# split data by index
split_idx = int(0.8 * len(shuffled))
train = shuffled.iloc[:split_idx].copy()
test = shuffled.iloc[split_idx:].copy()

In [6]:
# ============================================================
# 3. MODEL 2 STRUCTURE
# ============================================================
parents = {
    'age_bin': [],
    'race': [],
    'sex': [],
    'native-country': [],
    'education': ['age_bin', 'race', 'sex', 'native-country'],
    'occupation': ['education'],
    'workclass': ['education'],
    'hours_bin': ['occupation', 'workclass'],
    'marital-status': ['age_bin'],
    'income': ['marital-status', 'education', 'occupation', 'hours_bin']
}

# sanity check: keys in parents must match nodes
assert set(parents.keys()) == set(nodes)

# all possible states from TRAINING data
states = {col: sorted(train[col].unique())for col in nodes}
states

{'age_bin': ['Adult', 'Middle', 'Senior', 'Young'],
 'race': ['Amer-Indian-Eskimo',
  'Asian-Pac-Islander',
  'Black',
  'Other',
  'White'],
 'sex': ['Female', 'Male'],
 'native-country': ['Asia',
  'Europe',
  'Latin-America',
  'Missing',
  'Other',
  'US'],
 'education': ['Bachelors',
  'Grad',
  'HS-grad',
  'HS-or-less',
  'Some-college/Assoc'],
 'occupation': ['Adm-clerical',
  'Armed-Forces',
  'Craft-repair',
  'Exec-managerial',
  'Farming-fishing',
  'Handlers-cleaners',
  'Machine-op-inspct',
  'Missing',
  'Other-service',
  'Priv-house-serv',
  'Prof-specialty',
  'Protective-serv',
  'Sales',
  'Tech-support',
  'Transport-moving'],
 'workclass': ['Gov', 'Missing', 'Not-working', 'Private/Self-emp'],
 'hours_bin': ['Extreme', 'Full-time', 'Long', 'Part-time'],
 'marital-status': ['Divorced',
  'Married-AF-spouse',
  'Married-civ-spouse',
  'Married-spouse-absent',
  'Never-married',
  'Separated',
  'Widowed'],
 'income': ['<=50K', '>50K']}

In [7]:
# ============================================================
# 4. Learn CPTs with MLE + Laplace smoothing
# ============================================================
laplace = 1.0
CPTs = {}

def fit_cpt(child, parent_list, train_df, states_dict, laplace=1.0):
    """
    Fit CPT for P(child | parent) using MLE (with Laplace smoothing)
    
    Returns: dict parents -> {child_state: prob}
    where parents is either () (for root) or tuple of parent values
    """
    child_states = states_dict[child]
    cpt = {}
    
    # if root node, compute P(child)
    if len(parent_list) == 0:
        counts = train_df[child].value_counts()
        total = len(train_df)
        denom = total + laplace * len(child_states)
        cpt[()] = {}
        for cs in child_states:
            num = counts.get(cs, 0) + laplace
            cpt[()][cs] = num / denom
        return cpt
    
    # if non-root node, compute P(child | parents)
    parent_states_lists = [states_dict[p] for p in parent_list]
    
    for parent_cfg in product(*parent_states_lists):
        # filter rows matching parent configuration
        subset = train_df
        for p, v in zip(parent_list, parent_cfg):
            subset = subset[subset[p] == v]
        
        counts = subset[child].value_counts()
        total = len(subset)
        denom = total + laplace * len(child_states)
        
        cpt[parent_cfg] = {}
        for cs in child_states:
            num = counts.get(cs, 0) + laplace
            cpt[parent_cfg][cs] = num / denom

    return cpt

# fit CPT for each node
for node in nodes:
    CPTs[node] = fit_cpt(node, parents[node], train, states, laplace=laplace)

CPTs

# fit_cpt('age_bin', [], train, states)
# fit_cpt('education', ['age_bin', 'race', 'sex', 'native-country'], train, states)

{'age_bin': {(): {'Adult': np.float64(0.5041840468817975),
   'Middle': np.float64(0.2613557847327072),
   'Senior': np.float64(0.03705504516723392),
   'Young': np.float64(0.19740512321826137)}},
 'race': {(): {'Amer-Indian-Eskimo': np.float64(0.009954450074210553),
   'Asian-Pac-Islander': np.float64(0.03063104560110548),
   'Black': np.float64(0.09598751215517683),
   'Other': np.float64(0.007779313168534726),
   'White': np.float64(0.8556476790009724)}},
 'sex': {(): {'Female': np.float64(0.33172104926423546),
   'Male': np.float64(0.6682789507357646)}},
 'native-country': {(): {'Asia': np.float64(0.018859233859617696),
   'Europe': np.float64(0.01586529849791448),
   'Latin-America': np.float64(0.03950971109803219),
   'Missing': np.float64(0.01745182834770593),
   'Other': np.float64(0.01069628189052944),
   'US': np.float64(0.8976176463062002)}},
 'education': {('Adult',
   'Amer-Indian-Eskimo',
   'Female',
   'Asia'): {'Bachelors': 0.2, 'Grad': 0.2, 'HS-grad': 0.2, 'HS-or-less

In [8]:
# ============================================================
# 5. Compute log-likelihood, parameter count, and BIC
# ============================================================
def compute_loglike_and_params(data_df, CPTs, parents, states, nodes, sparse_threshold=1e-4):
    loglike = 0.0
    num_params = 0
    sparsity = {n: 0 for n in nodes}
    
    # parameter count
    for child in nodes:
        child_states = states[child]
        m = len(child_states)
        parent_list = parents[child]
        
        if len(parent_list) == 0:
            num_parent_cfg = 1
        else:
            num_parent_cfg = 1
            for p in parent_list:
                num_parent_cfg *= len(states[p])
        
        # for each parent config, (m-1) free parameters
        num_params += num_parent_cfg * (m - 1)
        
        # count sparse cells for diagnostics
        for parent_cfg, dist in CPTs[child].items():
            for p in dist.values():
                if p < sparse_threshold:
                    sparsity[child] += 1
    
    # log-likelihood on given data
    for _, row in data_df.iterrows():
        for child in nodes:
            parent_list = parents[child]
            
            if len(parent_list) == 0:
                parent_cfg = ()
            else:
                parent_cfg = tuple(row[p] for p in parent_list)

            p = CPTs[child][parent_cfg].get(row[child], 1e-12)
            if p <= 0:
                p = 1e-12
            loglike += math.log(p)
    
    return loglike, num_params, sparsity

loglike, params, sparsity = compute_loglike_and_params(train, CPTs, parents, states, nodes)
bic = -2 * loglike + params * math.log(len(train))

print(f"Log-likelihood (train): {loglike:.4f}")
print(f"Number of parameters: {params}")
print(f"BIC: {bic:.4f}")
print("Sparse CPT cells per node (p < 1e-4):")
for n in nodes:
    print(f"  {n}: {sparsity[n]}")

Log-likelihood (train): -370733.2301
Number of parameters: 3362
BIC: 777013.5148
Sparse CPT cells per node (p < 1e-4):
  age_bin: 0
  race: 0
  sex: 0
  native-country: 0
  education: 0
  occupation: 0
  workclass: 0
  hours_bin: 0
  marital-status: 0
  income: 0


In [10]:
test_preds = predict(test, CPTs, parents, states, nodes)

# Evaluate
from sklearn.metrics import accuracy_score, confusion_matrix

acc = accuracy_score(test["income"], test_preds)
cm  = confusion_matrix(test["income"], test_preds)

print("Accuracy:", acc)
print("Confusion Matrix:\n", cm)

Accuracy: 0.8293581738151294
Confusion Matrix:
 [[6871  555]
 [1112 1231]]


In [ ]:

def posterior_income_model2(row, CPTs, parents, states, nodes):
    income_states = states['income']
    scores = {}

    for inc in income_states:
        row_modified = row.copy()
        row_modified['income'] = inc

        inc_parents = parents['income']
        if len(inc_parents) == 0:
            parent_cfg = ()
        else:
            parent_cfg = tuple(row_modified[p] for p in inc_parents)

        p_inc = CPTs['income'][parent_cfg].get(inc, 1e-12)

        likelihood = score_row(row_modified, CPTs, parents, states, nodes)

    
        scores[inc] = p_inc * likelihood

    total = sum(scores.values())
    if total == 0:
      
        return {inc: 1.0 / len(income_states) for inc in income_states}

    return {inc: scores[inc] / total for inc in income_states}


example_post = posterior_income_model2(test.iloc[0], CPTs, parents, states, nodes)
print("Example posterior for first test row:", example_post)

def avg_p_gt50_given_model2(condition_fn):
    subset = test[condition_fn(test)]
    if len(subset) == 0:
        return 0, None

    probs = []
    for _, row in subset.iterrows():
        post = posterior_income_model2(row, CPTs, parents, states, nodes)
        probs.append(post['>50K'])

    return len(subset), sum(probs) / len(probs)

n_bach, p_bach = avg_p_gt50_given_model2(
    lambda df: df['education'] == 'Bachelors'
)

n_grad, p_grad = avg_p_gt50_given_model2(
    lambda df: df['education'].isin(['Masters', 'Doctorate', 'Prof-school'])
)

n_ft, p_ft = avg_p_gt50_given_model2(
    lambda df: df['hours_bin'] == 'Full-time'
)

n_ext, p_ext = avg_p_gt50_given_model2(
    lambda df: df['hours_bin'] == 'Extreme'
)

n_exec, p_exec = avg_p_gt50_given_model2(
    lambda df: df['occupation'] == 'Exec-managerial'
)

print("\nModel 2 posterior averages:")
print("education = Bachelors           -> n =", n_bach,  "  P(>50K) =", p_bach)
print("education = Grad (M/Dr/Prof)    -> n =", n_grad,  "  P(>50K) =", p_grad)
print("hours_bin = Full-time           -> n =", n_ft,    "  P(>50K) =", p_ft)
print("hours_bin = Extreme             -> n =", n_ext,   "  P(>50K) =", p_ext)
print("occupation = Exec-managerial    -> n =", n_exec,  "  P(>50K) =", p_exec)

Example posterior for first test row: {'<=50K': np.float64(0.5714285714285715), '>50K': np.float64(0.4285714285714286)}

Model 2 posterior averages:
education = Bachelors           -> n = 1633   P(>50K) = 0.4244311707553269
education = Grad (M/Dr/Prof)    -> n = 0   P(>50K) = None
hours_bin = Full-time           -> n = 5389   P(>50K) = 0.21410728477137653
hours_bin = Extreme             -> n = 322   P(>50K) = 0.40647315743359197
occupation = Exec-managerial    -> n = 1211   P(>50K) = 0.4843810998223131
